# Tableau 
* To-Do: 
    * Move function outside of project_size outside
    * Add project categorizing 

In [1]:
import A1_data_prep
import A2_tableau
import A5_crosswalks as crosswalks
import numpy as np
import pandas as pd
from calitp import *

pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [13]:
def project_size_rating(dataframe, original_column:str, new_column:str):
    """Rate a project by big, small, and medium

    Extended description of function.

    Args:
        dataframe
        original_column (str): column to create the metric off of
        new_column (str): new column to hold results

    Returns:
        bool: Description of return value

    """
    # Get percentiles in objects for total vehicle.
    p75 = dataframe[original_column].quantile(0.75).astype(float)
    p25 = dataframe[original_column].quantile(0.25).astype(float)
    p50 = dataframe[original_column].quantile(0.50).astype(float)

    # Function for fleet size
    def project_size(row):
        if (row[original_column] > 0) and (row[original_column] <= p25):
            return "Small"
        elif (row[original_column] > p25) and (row[original_column] <= p75):
            return "Medium"
        elif row[original_column] > p75:
            return "Large"
        else:
            return "No Info"

    dataframe[new_column] = dataframe.apply(lambda x: fleet_size(x), axis=1)

    return dataframe

In [14]:
# Categorizing expended percentage into bins
def expended_percent(row):
    if (row.Expended_Percent > 0) and (row.Expended_Percent < 0.26):
        return "1-25"
    elif (row.Expended_Percent > 0.25) and (row.Expended_Percent < 0.51):
        return "26-50"
    elif (row.Expended_Percent > 0.50) and (row.Expended_Percent < 0.76):
        return "51-75"
    elif (row.Expended_Percent > 0.75) and (row.Expended_Percent < 0.95):
        return "76-99"
    elif row.Expended_Percent == 0.0:
        return "0"
    else:
        return "100"


# Categorize years and expended_percent_group into bins
def progress(df):
    # 2015
    if (df["project_award_year"] == 2015) and (
        df["Expended_Percent_Group"] == "1-25"
    ) | (df["Expended_Percent_Group"] == "26-50"):
        return "Behind"
    elif (df["project_award_year"] == 2015) and (
        df["Expended_Percent_Group"] == "76-99"
    ) | (df["Expended_Percent_Group"] == "51-75"):
        return "On Track"

    # 2016
    elif (df["project_award_year"] == 2016) and (
        df["Expended_Percent_Group"] == "1-25"
    ) | (df["Expended_Percent_Group"] == "26-50"):
        return "Behind"
    elif (df["project_award_year"] == 2016) and (
        df["Expended_Percent_Group"] == "51-75"
    ) | (df["Expended_Percent_Group"] == "76-99"):
        return "On Track"

    # 2018
    elif (df["project_award_year"] == 2018) and (
        df["Expended_Percent_Group"] == "1-25"
    ):
        return "Behind"
    elif (df["project_award_year"] == 2018) and (
        df["Expended_Percent_Group"] == "26-50"
    ) | (df["Expended_Percent_Group"] == "51-75"):
        return "On Track"
    elif (df["project_award_year"] == 2018) and (
        df["Expended_Percent_Group"] == "76-99"
    ):
        return "Ahead"

    # 2020
    elif (df["project_award_year"] == 2020) and (
        df["Expended_Percent_Group"] == "1-25"
    ):
        return "Behind"
    elif (df["project_award_year"] == 2020) and (
        df["Expended_Percent_Group"] == "26-50"
    ):
        return "On Track"
    elif (df["project_award_year"] == 2020) and (
        df["Expended_Percent_Group"] == "51-75"
    ) | (df["Expended_Percent_Group"] == "76-99"):
        return "Ahead"

    # 0 Expenditures
    elif df["Expended_Percent_Group"] == "0":
        return "No expenditures recorded"

    else:
        return "100% of allocated funds spent"

In [15]:
def tableau():
    # Load in cleaned project sheets
    df = A1_data_prep.clean_project()

    # Replace districts & counties with their full names
    df["project_district"] = df["project_district"].replace(crosswalks.full_ct_district)
    df["project_county"] = df["project_county"].replace(crosswalks.full_county)
    
    # Create new cols
    df = df.assign(
        Expended_Percent=(
            df["project_expended_amount"] / df["project_allocated_amount"]
        ).fillna(0),
        Allocated_Percent=(
            df["project_allocated_amount"] / df["project_tircp_award_amount__$_"]
        ).fillna(0),
        Unallocated_Amount=(
            df["project_tircp_award_amount__$_"] - df["project_allocated_amount"]
        ).fillna(0),
        Projects_Funded_Percent=(
            df["project_tircp_award_amount__$_"] / df["project_total_project_cost"]
        ).fillna(0),
    )
    
    """
    Categorize projects whether they are ahead/behind/0 expenditures/etc
    """
    # Categorize projects first into expended % bins
    df["Expended_Percent_Group"] = df.apply(lambda x: expended_percent(x), axis=1)

    # Second, apply progress function
    df["Progress"] = df.apply(progress, axis=1)

    # Categorize projects whether they are large/small/med based on TIRCP amount
    df = df.rename(columns={"project_tircp_award_amount__$_": "tircp"})

    """
    Categorize projects by project size
    """
    df = project_size_rating(df, "tircp", "Project_Category")

    # Clean Up Column Names
    df = A1_data_prep.clean_up_columns(df)

    """
    # Write to GCS
    with pd.ExcelWriter(f"{GCS_FILE_PATH}Script_Tableau_Sheet.xlsx") as writer:
        df.to_excel(writer, sheet_name="Data", index=False)
    """
    return df

In [16]:
tableau_test = tableau()

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
/tmp/ipykernel_3388/2937170163.py:4: FutureWarning: The default value of regex will change from True to False in a future version.


In [19]:
tableau_test[
    [
        "Award Year",
        "Grant Recipient",
        "Description",
        "Expended Percent",
        "Progress",
        "Project Category",
    ]
].head(2)

,Award Year,Grant Recipient,Description,Expended Percent,Progress,Project Category
0,2015,Antelope Valley Transit Authority (AVTA),Purchase 13 60-foot articulated BRT buses and 16 45-foot electric commuter buses,0.889816,On Track,Medium
1,2015,Capitol Corridor Joint Powers Authority,"Track and curve improvements between San Jose and Martinez for faster journeys benefiting Capitol Corridor, ACE, and San Joaquins passengers",1.000000,100% of allocated funds spent,Small
